Parts of the code were inspired by this [BERT Tutorial](https://towardsdatascience.com/fine-tuning-bert-for-text-classification-54e7df642894#96e0) <br>
The EAR technique (i.e. the function def compute_negative_entropy) has been implemented according to the [GitHub repository](https://github.com/g8a9/ear) hosting the code associated with the original [EAR paper](https://aclanthology.org/2022.findings-acl.88/) by Attanasio et al.

In [ ]:
! pip install transformers datasets evaluate
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tqdm import trange
import random

In [ ]:
#!pip list #get the version of the installed packages
#!python --version #get the python version

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')
train_df = pd.read_csv('/content/drive/My Drive/DATASETS/wiki_toxic/train.csv')
validation_df = pd.read_csv('/content/drive/My Drive/DATASETS/wiki_toxic/validation.csv')
frac = 0.5

#TRAIN
print(train_df.shape[0]) # get the number of rows in the dataframe
rows_to_delete = train_df.sample(frac=frac, random_state=1) # randomly select half of the rows. Random_state ensures reproducibility
train_df = train_df.drop(rows_to_delete.index)
print(train_df.shape[0])

#VALIDATION
print(validation_df.shape[0]) # get the number of rows in the dataframe
rows_to_delete = validation_df.sample(frac=frac, random_state=1) # randomly select half of the rows. Random_state ensures reproducibility
validation_df = validation_df.drop(rows_to_delete.index)
print(validation_df.shape[0])

train_text = train_df.comment_text.values
train_labels = train_df.label.values
validation_text = validation_df.comment_text.values
validation_labels = validation_df.label.values

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')
#TEST
test_df = pd.read_csv('/content/drive/My Drive/DATASETS/wiki_toxic/test.csv')
print(test_df.shape[0]) # get the number of rows in the dataframe
test_text = test_df.comment_text.values
test_labels = test_df.label.values

In [ ]:
#@title
def compute_negative_entropy(
    inputs: tuple, attention_mask: torch.Tensor, return_values=False
):
    """Compute the negative entropy across layers of a network for given inputs.

    Args:
        - input: tuple. Tuple of length num_layers. Each item should be in the form: BHSS
        - attention_mask. Tensor with dim: BS
    """
    inputs = torch.stack(inputs)  #  LayersBatchHeadsSeqlenSeqlen
    assert inputs.ndim == 5, "Here we expect 5 dimensions in the form LBHSS"

    #  average over attention heads
    pool_heads = inputs.mean(2)

    batch_size = pool_heads.shape[1]
    samples_entropy = list()
    neg_entropies = list()
    for b in range(batch_size):
        #  get inputs from non-padded tokens of the current sample
        mask = attention_mask[b]
        sample = pool_heads[:, b, mask.bool(), :]
        sample = sample[:, :, mask.bool()]

        #  get the negative entropy for each non-padded token
        neg_entropy = (sample.softmax(-1) * sample.log_softmax(-1)).sum(-1)
        if return_values:
            neg_entropies.append(neg_entropy.detach())

        #  get the "average entropy" that traverses the layer
        mean_entropy = neg_entropy.mean(-1)

        #  store the sum across all the layers
        samples_entropy.append(mean_entropy.sum(0))

    # average over the batch
    final_entropy = torch.stack(samples_entropy).mean()
    if return_values:
        return final_entropy, neg_entropies
    else:
        return final_entropy

In [ ]:
#@title
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_token_id = []
train_attention_masks = []
validation_token_id = []
validation_attention_masks = []

def preprocessing(input_text, tokenizer):
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 250,
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in train_text:
  encoding_dict = preprocessing(sample, tokenizer)
  train_token_id.append(encoding_dict['input_ids'])
  train_attention_masks.append(encoding_dict['attention_mask'])
train_token_id = torch.cat(train_token_id, dim = 0)
train_attention_masks = torch.cat(train_attention_masks, dim = 0)
train_labels = torch.tensor(train_labels)

for sample in validation_text:
  encoding_dict = preprocessing(sample, tokenizer)
  validation_token_id.append(encoding_dict['input_ids'])
  validation_attention_masks.append(encoding_dict['attention_mask'])
validation_token_id = torch.cat(validation_token_id, dim = 0)
validation_attention_masks = torch.cat(validation_attention_masks, dim = 0)
validation_labels = torch.tensor(validation_labels)

In [ ]:
#@title
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 32

train_idx = np.arange(len(train_labels))
val_idx = np.arange(len(validation_labels))

# Train and validation sets
train_set = TensorDataset(train_token_id[train_idx],
                          train_attention_masks[train_idx],
                          train_labels[train_idx])

val_set = TensorDataset(validation_token_id[val_idx],
                        validation_attention_masks[val_idx],
                        validation_labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
#@title
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  return b_precision, b_recall

In [ ]:
#@title
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = True,
    output_hidden_states = False,
)

model.config.problem_type = "single_label_classification" #in this way Cross Entropy loss is selected

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(),
                              lr = 2e-5,
                              weight_decay=0.01,
                              )

# Run on GPU
model.cuda()

In [ ]:
#@title
device = torch.device('cuda')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids,
                             token_type_ids = None,
                             attention_mask = b_input_mask,
                             labels = b_labels)

        reg_strength = 0.0 #tweak this parameter to apply regularisation. reg_strength = 0.01
        neg_entropy = compute_negative_entropy(
            inputs=train_output.attentions,
            attention_mask=b_input_mask
        )
        reg_loss = reg_strength * neg_entropy
        loss = train_output.loss + reg_loss

        # Backward pass
        loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_precision = []
    val_recall = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids,
                              token_type_ids = None,
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_precision, b_recall = b_metrics(logits, label_ids)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    precision = sum(val_precision)/len(val_precision)
    recall = sum(val_recall)/len(val_recall)
    f1_score = 2*((precision*recall)/(precision+recall))
    print('\t - Validation Precision: {:.4f}'.format(precision) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(recall) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation F1-score: {:.4f}'.format(f1_score) if (precision+recall)!=0 else '\t - Validation F1-score: NaN')

In [ ]:
#@title
model_save_name = 'FINAL_bert_ear_reg_0_00.bin'
path = F"drive/My Drive/MODELS/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
#@title
#LOAD MODEL
from google.colab import drive
import pandas as pd
pd.options.mode.chained_assignment = None
drive.mount('/content/drive')
model_save_name = 'FINAL_bert_ear_reg_0_01.bin'
path = F"drive/My Drive/MODELS/{model_save_name}"
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = True,
    output_hidden_states = False,
)
device = torch.device('cuda')
model.load_state_dict(torch.load(path,map_location=device))
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocessing(input_text, tokenizer):
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 250,
                        truncation=True,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [ ]:
#@title
def classifier(comment:str):
  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []
  # Apply the tokenizer
  encoding = preprocessing(comment, tokenizer)
  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)
  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))
  prediction = 'TOXIC' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'NON-TOXIC'
  return prediction

In [ ]:
#@title
print(classifier("I am african"))
print(classifier("I love african people"))
print(classifier("I hate african people"))
print('-')
print(classifier("I am blind"))
print(classifier("I love blind people"))
print(classifier("I hate blind people"))

## Compute F1-score on test dataset

In [ ]:
from sklearn.metrics import f1_score
predictions = []
for comment_text in test_text:
  result = classifier(comment_text)
  if(result=='TOXIC'):
    predictions.append(1)
  else:
    predictions.append(0)
print("F1-score on test dataset (BERT 0_01): "+str(f1_score(test_labels,predictions)))

## Synthetic dataset pre-processing

In [ ]:
from sklearn.metrics import roc_auc_score
synthetic_test_df = pd.read_csv('/content/drive/My Drive/DATASETS/en_sentence_templates.csv')
#All the identities considered in EAR paper
list_identities = ["lesbian", "gay", "bisexual","transgender", "trans", "queer", "lgbt","lgbtq", "homosexual", "straight", "heterosexual","male", "female", "nonbinary", "african","african american", "black", "white", "european","hispanic", "latino", "latina", "latinx", "mexican","canadian", "american", "asian", "indian","middle eastern", "chinese", "japanese", "christian","muslim", "jewish", "buddhist", "catholic","protestant", "sikh", "taoist", "old", "older", "young","younger", "teenage", "millenial", "middle aged","elderly", "blind", "deaf", "paralyzed"]
print(len(list_identities))
#A sub_list of all the identities
gender_sub_list_identities = ["lesbian", "gay", "bisexual","transgender", "trans", "queer", "lgbt","lgbtq", "homosexual", "straight", "heterosexual","male", "female", "nonbinary"]
ethnicity_sub_list_identities = ["african","african american", "black", "white", "european","hispanic", "latino", "latina", "latinx", "mexican","canadian", "american", "asian", "indian","middle eastern", "chinese", "japanese"]
religion_sub_list_identities = ["christian","muslim", "jewish", "buddhist", "catholic","protestant", "sikh", "taoist"]
age_sub_list_identities = ["old", "older", "young","younger", "teenage", "millenial", "middle aged","elderly", "blind", "deaf", "paralyzed"]
print(len(age_sub_list_identities))

In [ ]:
#@title
print(synthetic_test_df.shape[0])
#drop some random names
name_drop = ['Êú™Êú™','Zoe','Ziv','Ying','Thel','Sulin','Sooyin','Sohaib','Sara','Santiago','Ram√≥n','Nithum','Myesha','Mukhtar','Morgan','Mohammed','Moeta','Medina','Mateo','Mahir','Lucas','ÏÑúÏú§','Abdul','Agust√≠n','Aishah','Aki','Alva','Anastasiya','Angela','Anita','Baratunde','Binh','Chris','Dlshad','Elizabeth','Emad','Hiro','Hoshi','Ioannis','Javeria','Jim','Joe','L√©','Liam','Ling']
# drop the rows that contain the string name in the 'phrase' column
for name in name_drop:
  synthetic_test_df = synthetic_test_df[~synthetic_test_df['phrase'].str.contains(name)]
print(synthetic_test_df.shape[0])

In [ ]:
# drop_identities = list_identities - sub_list_identities
drop_identities = [x for x in list_identities if x not in age_sub_list_identities]
for identity in drop_identities:
  # drop the rows that contain the string identity in the 'phrase' column
  synthetic_test_df = synthetic_test_df[~synthetic_test_df['phrase'].str.contains(identity)]
print(synthetic_test_df.shape[0])

## Compute AUC metrics

In [ ]:
def compute_auc(df:pd.DataFrame):
  predictions = []
  list_comments = list(df['phrase'])
  for comment in list_comments:
    result = classifier(comment)
    if(result=='TOXIC'):
      predictions.append(1)
    else:
      predictions.append(0)
  print(predictions)
  df["toxicity"] = df["toxicity"].map({"nontoxic": 0, "toxic": 1})
  return roc_auc_score(df['toxicity'],predictions)

In [ ]:
metrics_results = {'subgroup_AUC':[],'BPSN_AUC':[],'BNSP_AUC':[]}
for identity in gender_sub_list_identities:
  print(identity)
  subgroup_auc_df = synthetic_test_df.loc[synthetic_test_df['phrase'].str.contains(identity)].copy()
  score = compute_auc(subgroup_auc_df)
  metrics_results['subgroup_AUC'].append((identity,score))
  BPSN_auc_df = synthetic_test_df.loc[((synthetic_test_df['phrase'].str.contains(identity)) & (synthetic_test_df['toxicity'] == 'nontoxic')) | (~(synthetic_test_df['phrase'].str.contains(identity)) & (synthetic_test_df['toxicity'] == 'toxic'))].copy()
  score = compute_auc(BPSN_auc_df)
  metrics_results['BPSN_AUC'].append((identity,score))
  BNSP_auc_df = synthetic_test_df.loc[((synthetic_test_df['phrase'].str.contains(identity)) & (synthetic_test_df['toxicity'] == 'toxic')) | (~(synthetic_test_df['phrase'].str.contains(identity)) & (synthetic_test_df['toxicity'] == 'nontoxic'))].copy()
  score = compute_auc(BNSP_auc_df)
  metrics_results['BNSP_AUC'].append((identity,score))

In [ ]:
def compute_avg_scores(AUC_dict:dict):
  #print(AUC_dict)
  for metric in AUC_dict:
    sum=0
    for tup in AUC_dict[metric]:
      sum += tup[1]
    average = sum/len(AUC_dict[metric])
    print('Avg '+metric+': '+str(round(average, 10)))

In [ ]:
compute_avg_scores(metrics_results)

## Compute F1-score on synthetic dataset

In [ ]:
from sklearn.metrics import f1_score
synthetic_comments = list(synthetic_test_df['phrase'])
synthetic_labels = synthetic_test_df["toxicity"].map({"nontoxic": 0, "toxic": 1})
predictions = []
for comment in synthetic_comments:
  result = classifier(comment)
  if(result=='TOXIC'):
    predictions.append(1)
  else:
    predictions.append(0)
print("F1-score on synthetic dataset (BERT 0_00): "+str(f1_score(synthetic_labels,predictions)))

## Compute Avg AUC scores for all identities

In [ ]:
FINAL_bert_ear_reg_0_00 = {'subgroup_AUC': [('lesbian', 0.9172932330827068), ('gay', 0.5563909774436091), ('bisexual', 0.9962406015037594), ('transgender', 0.9210526315789473), ('trans', 0.9454887218045113), ('queer', 0.8609022556390977), ('lgbt', 0.9661654135338346), ('lgbtq', 0.9360902255639098), ('homosexual', 0.5075187969924813), ('straight', 0.9624060150375939), ('heterosexual', 0.8609022556390977), ('male', 0.9887218045112782), ('female', 0.9887218045112782), ('nonbinary', 0.924812030075188), ('african', 0.9718045112781954), ('african american', 0.9624060150375939), ('black', 0.9661654135338346), ('white', 0.9699248120300752), ('european', 0.981203007518797), ('hispanic', 0.9586466165413534), ('latino', 0.9774436090225564), ('latina', 0.8909774436090225), ('latinx', 0.9473684210526316), ('mexican', 1.0), ('canadian', 0.9624060150375939), ('american', 0.9736842105263157), ('asian', 0.9511278195488722), ('indian', 0.9849624060150376), ('middle eastern', 0.887218045112782), ('chinese', 0.9849624060150376), ('japanese', 0.981203007518797), ('christian', 0.9060150375939849), ('muslim', 0.9924812030075187), ('jewish', 1.0), ('buddhist', 0.9774436090225564), ('catholic', 0.887218045112782), ('protestant', 0.9398496240601504), ('sikh', 0.9699248120300752), ('taoist', 0.9473684210526316), ('old', 0.9172932330827068), ('older', 0.8947368421052632), ('young', 0.9210526315789473), ('younger', 0.9097744360902256), ('teenage', 0.9887218045112782), ('millenial', 0.8646616541353384), ('middle aged', 0.9398496240601504), ('elderly', 0.9661654135338346), ('blind', 0.9887218045112781), ('deaf', 0.7330827067669172), ('paralyzed', 0.981203007518797)], 'BPSN_AUC': [('lesbian', 0.8987854251012146), ('gay', 0.5378831694621169), ('bisexual', 0.9817813765182186), ('transgender', 0.9025448235974551), ('trans', 0.9429824561403508), ('queer', 0.8423944476576055), ('lgbt', 0.9855889724310777), ('lgbtq', 0.9864083285135916), ('homosexual', 0.489010989010989), ('straight', 0.9843840370156159), ('heterosexual', 0.8464430306535569), ('male', 0.981829573934837), ('female', 0.9823597455176403), ('nonbinary', 0.9872758820127241), ('african', 0.9641604010025062), ('african american', 0.9652255639097744), ('black', 0.9290413533834586), ('white', 0.9647556390977443), ('european', 0.9640507518796992), ('hispanic', 0.9654605263157895), ('latino', 0.9642857142857143), ('latina', 0.9696898496240601), ('latinx', 0.9661654135338346), ('mexican', 0.962875939849624), ('canadian', 0.9652255639097744), (
    'american', 0.9639097744360903), ('asian', 0.9659304511278195), ('indian', 0.9638157894736843), ('middle eastern', 0.9699248120300752), ('chinese', 0.9638157894736843), ('japanese', 0.9640507518796992), ('christian', 0.9597207303974221), ('muslim', 0.9430719656283565), ('jewish', 0.9462943071965628), ('buddhist', 0.949516648764769), ('catholic', 0.9624060150375939), ('protestant', 0.9548872180451128), ('sikh', 0.9505907626208379), ('taoist', 0.9538131041890441), ('old', 0.9532163742690059), ('older', 0.9518796992481203), ('young', 0.9523809523809523), ('younger', 0.9503759398496241), ('teenage', 0.9424812030075188), ('millenial', 0.9548872180451128), ('middle aged', 0.9473684210526316), ('elderly', 0.9447368421052631), ('blind', 0.9342105263157894), ('deaf', 0.674436090225564), ('paralyzed', 0.943233082706767)], 'BNSP_AUC': [('lesbian', 0.9008097165991903), ('gay', 0.9285714285714286), ('bisexual', 0.8906882591093117), ('transgender', 0.9005205320994795), ('trans', 0.8771929824561403), ('queer', 0.9051474840948526), ('lgbt', 0.8518170426065163), ('lgbtq', 0.8305378831694621), ('homosexual', 0.9323308270676691), ('straight', 0.8568536726431463), ('heterosexual', 0.9010989010989011), ('male', 0.8743734335839599), ('female', 0.8831694621168305), ('nonbinary', 0.8192596876807403), ('african', 0.969548872180451), ('african american', 0.9602913533834586), ('black', 1.0), ('white', 0.9678101503759399), ('european', 0.9790883458646616), ('hispanic', 0.9565319548872182), ('latino', 0.9753289473684211), ('latina', 0.8888627819548872), ('latinx', 0.9452537593984963), ('mexican', 0.9978853383458647), ('canadian', 0.9602913533834586), ('american', 0.9714285714285714), ('asian', 0.9490131578947368), ('indian', 0.9828477443609023), ('middle eastern', 0.8851033834586467), ('chinese', 0.9828477443609023), ('japanese', 0.9790883458646616), ('christian', 0.9054779806659506), ('muslim', 0.9962406015037594), ('jewish', 0.9994629430719657), ('buddhist', 0.9769065520945222), ('catholic', 0.8866809881847477), ('protestant', 0.9393125671321161), ('sikh', 0.9693877551020408), ('taoist', 0.9468313641245972), ('old', 0.8868003341687551), ('older', 0.8672932330827068), ('young', 0.8905597326649958), ('younger', 0.8823308270676692), ('teenage', 0.9612781954887218), ('millenial', 0.837218045112782), ('middle aged', 0.912406015037594), ('elderly', 0.9387218045112782), ('blind', 0.969548872180451), ('deaf', 0.9992481203007519), ('paralyzed', 0.9537593984962406)]}

FINAL_bert_ear_reg_0_01 = {'subgroup_AUC': [('lesbian', 0.8909774436090225), ('gay', 0.6428571428571428), ('bisexual', 0.9774436090225563), ('transgender', 0.9398496240601503), ('trans', 0.9530075187969925), ('queer', 0.5338345864661654), ('lgbt', 0.9868421052631579), ('lgbtq', 0.981203007518797), ('homosexual', 0.5037593984962406), ('straight', 0.9586466165413534), ('heterosexual', 0.9285714285714285), ('male', 0.9924812030075187), ('female', 0.9924812030075187), ('nonbinary', 0.9548872180451128), ('african', 0.9943609022556391), ('african american', 0.9962406015037594), ('black', 0.9548872180451128), ('white', 0.9774436090225563), ('european', 0.9699248120300752), ('hispanic', 0.9887218045112782), ('latino', 0.9849624060150376), ('latina', 0.9849624060150376), ('latinx', 0.981203007518797), ('mexican', 0.9887218045112781), ('canadian', 0.9736842105263157), ('american', 0.9943609022556391), ('asian', 0.9887218045112781), ('indian', 0.9887218045112781), ('middle eastern', 0.981203007518797), ('chinese', 0.9962406015037594), ('japanese', 0.9887218045112782), ('christian', 0.981203007518797), ('muslim', 0.9924812030075187), ('jewish', 0.9924812030075187), ('buddhist', 0.9887218045112782), ('catholic', 0.9924812030075187), ('protestant', 0.9849624060150376), ('sikh', 0.9887218045112782), ('taoist', 0.9774436090225564), ('old', 0.9417293233082705), ('older', 0.9135338345864661), ('young', 0.9492481203007519), ('younger', 0.9323308270676691), ('teenage', 0.9887218045112781), ('millenial', 0.9060150375939849), ('middle aged', 0.9849624060150376), ('elderly', 0.981203007518797), ('blind', 0.9360902255639098), ('deaf', 0.868421052631579), ('paralyzed', 0.9285714285714286)], 'BPSN_AUC': [('lesbian', 0.8779641411220357), ('gay', 0.6298438403701562), ('bisexual', 0.9725274725274725), ('transgender', 0.9349334875650666), ('trans', 0.9630325814536342), ('queer', 0.5208212839791787), ('lgbt', 0.9859022556390976), ('lgbtq', 0.9884326200115674), ('homosexual', 0.49074609600925395), ('straight', 0.9901677270098324), ('heterosexual', 0.9196067090803932), ('male', 0.9827694235588972), ('female', 0.9835164835164835), ('nonbinary', 0.9904569115095431), ('african', 0.982456140350877), ('african american', 0.9830827067669173), ('black', 0.9537124060150376), ('white', 0.9842575187969925), ('european', 0.9887218045112782), ('hispanic', 0.987546992481203), ('latino', 0.987781954887218), ('latina', 0.987781954887218), ('latinx', 0.9880169172932332), ('mexican', 0.9835526315789473), (
    'canadian', 0.9884868421052632), ('american', 0.9845864661654136), ('asian', 0.9835526315789473), ('indian', 0.9835526315789473), ('middle eastern', 0.9880169172932332), ('chinese', 0.9830827067669173), ('japanese', 0.987546992481203), ('christian', 0.9892588614393125), ('muslim', 0.9833512352309344), ('jewish', 0.9833512352309344), ('buddhist', 0.9881847475832438), ('catholic', 0.9876476906552094), ('protestant', 0.9887218045112782), ('sikh', 0.9881847475832438), ('taoist', 0.9897959183673469), ('old', 0.9678362573099415), ('older', 0.9721804511278196), ('young', 0.9707602339181287), ('younger', 0.9703007518796993), ('teenage', 0.9605263157894736), ('millenial', 0.9729323308270676), ('middle aged', 0.9650375939849625), ('elderly', 0.9654135338345864), ('blind', 0.9037593984962407), ('deaf', 0.8319548872180451), ('paralyzed', 0.9086466165413535)], 'BNSP_AUC': [('lesbian', 0.887218045112782), ('gay', 0.9063042220936958), ('bisexual', 0.8724696356275303), ('transgender', 0.8753614806246386), ('trans', 0.8524436090225563), ('queer', 0.9146905725853095), ('lgbt', 0.8577694235588973), ('lgbtq', 0.8600347021399652), ('homosexual', 0.917004048582996), ('straight', 0.8374783111625217), ('heterosexual', 0.8802776171197224), ('male', 0.8656015037593985), ('female', 0.8753614806246385), ('nonbinary', 0.8337189126662811), ('african', 0.9946115288220552), ('african american', 0.9964755639097744), ('black', 0.9870770676691729), ('white', 0.9776785714285714), ('european', 0.9661654135338346), ('hispanic', 0.9849624060150376), ('latino', 0.9812030075187969), ('latina', 0.9812030075187969), ('latinx', 0.9774436090225563), ('mexican', 0.9889567669172932), ('canadian', 0.969924812030075), ('american', 0.9924812030075187), ('asian', 0.9889567669172932), ('indian', 0.9889567669172932), ('middle eastern', 0.9774436090225563), ('chinese', 0.9964755639097744), ('japanese', 0.9849624060150376), ('christian', 0.9801288936627282), ('muslim', 0.9957035445757251), ('jewish', 0.9957035445757251), ('buddhist', 0.9876476906552095), ('catholic', 0.9914070891514499), ('protestant', 0.9838882921589688), ('sikh', 0.9876476906552095), ('taoist', 0.9763694951664877), ('old', 0.9175020885547202), ('older', 0.8875939849624059), ('young', 0.9204260651629073), ('younger', 0.906390977443609), ('teenage', 0.9669172932330827), ('millenial', 0.8800751879699248), ('middle aged', 0.9590225563909774), ('elderly', 0.9552631578947368), ('blind', 0.9763157894736841), ('deaf', 0.9872180451127819), ('paralyzed', 0.9646616541353382)]}

print("FINAL_bert_ear_reg_0_00")
compute_avg_scores(FINAL_bert_ear_reg_0_00)
print("FINAL_bert_ear_reg_0_01")
compute_avg_scores(FINAL_bert_ear_reg_0_01)